# EXTRACTING FRAMES FROM A VIDEO FILE
---

This notebook lets you extract the frames of a given video file at the desired framerate and save them with numbered filenames as `.jpg` files in a new directory, keeping their native resolution.

First, let’s import the required librairies (openCV, numpy, os, math, datetime…)
> *(If you’re executing this notebook locally, you should install those with pip)*

In [6]:
import cv2
import numpy as np
import os
import math
#from datetime import timedelta
import datetime

Then, specify the path to the video file you’d like to extract the frames from.  

In [11]:
# enter file path to video (remote or local)
print("> Enter path to file")
video_file = str(input())
#ie. "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4"

# get the video frame rate
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# get the video duration (by dividing the total amount of frames by the video frame rate)
clip_duration = str(datetime.timedelta(seconds=int(frames/fps)))

if fps!=0:
  print("> This video contains", frames,"frames, sampled at", fps,"fps, for a total length of", clip_duration)
else:
  print("> No valid video found at this path")

> Enter path to file
http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4
> This video contains 360 frames, sampled at 23.976023976023978 fps, for a total length of 0:00:15 .


In [20]:
print("> Enter the frame rate at which you'd like to extract")
SAVING_FRAMES_PER_SECOND = float(input())
if SAVING_FRAMES_PER_SECOND<=fps:
  print("> At", SAVING_FRAMES_PER_SECOND, "FPS,", int(SAVING_FRAMES_PER_SECOND * (frames/fps)), "frames will be extracted in total.")
else:
  saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
  print("> /!\ The frame rate cannot be higher than the video's current rate. FPS set at", saving_frames_per_second)
# ie. 10

> Enter the frame rate at which you'd like to extract
2.5
> At 2.5 FPS, 37 frames will be extracted in total.
